This wrapper is needed to enable use of the Fortran cuBLAS matmul program to run as a shared library in python. Once loaded, you can run gpu_matmul(a,b) and get 12Tflops vs 4Tflops.

In [1]:
import ctypes
import numpy as np
import cupy as cp

# Load the Fortran shared library
_lib = ctypes.CDLL('./fast_matmul.so')

# Define the function prototype
_lib.tf32_matmul.argtypes = [
    ctypes.c_void_p,  # A
    ctypes.c_void_p,  # B
    ctypes.c_void_p,  # C
    ctypes.c_int,     # M
    ctypes.c_int,     # N
    ctypes.c_int      # K
]

def gpu_matmul(a, b):
    """
    Fast matrix multiplication using cuBLAS TF32.
    """
    # Convert inputs to float32 if needed
    if isinstance(a, np.ndarray):
        a = cp.asarray(a, dtype=np.float32)
    if isinstance(b, np.ndarray):
        b = cp.asarray(b, dtype=np.float32)
    
    # Ensure float32
    a = cp.asarray(a, dtype=np.float32)
    b = cp.asarray(b, dtype=np.float32)
    
    # Get dimensions
    M, K = a.shape
    K2, N = b.shape
    assert K == K2, "Inner dimensions must match"
    
    # Create output array
    c = cp.empty((M, N), dtype=np.float32)
    
    # Call Fortran function
    _lib.tf32_matmul(
        ctypes.c_void_p(a.data.ptr),
        ctypes.c_void_p(b.data.ptr),
        ctypes.c_void_p(c.data.ptr),
        M, N, K
    )
    
    return c


In [2]:
if __name__ == "__main__":
    # Create test matrices
    a = np.random.rand(10000, 10000).astype(np.float32)
    b = np.random.rand(10000, 10000).astype(np.float32)
    
    # Using our fast matmul
    c_fast = gpu_matmul(a, b)
    
    # Compare with cupy
    a_cp = cp.asarray(a)
    b_cp = cp.asarray(b)
    c_cp = cp.matmul(a_cp, b_cp)
    
    # Check results
    print("Max difference:", cp.max(cp.abs(c_fast - c_cp)))
    
    # Benchmark
    import time
    
    # Warm up
    for _ in range(5):
        c_fast = gpu_matmul(a_cp, b_cp)
        c_cp = cp.matmul(a_cp, b_cp)
    cp.cuda.Stream.null.synchronize()
    
    # Time our implementation
    t0 = time.perf_counter()
    for _ in range(10):
        c_fast = gpu_matmul(a_cp, b_cp)
    cp.cuda.Stream.null.synchronize()
    t1 = time.perf_counter()
    print(f"Fast matmul: {(t1-t0)/10*1000:.2f} ms")
    
    # Time cupy
    t0 = time.perf_counter()
    for _ in range(10):
        c_cp = cp.matmul(a_cp, b_cp)
    cp.cuda.Stream.null.synchronize()
    t1 = time.perf_counter()
    print(f"CuPy matmul: {(t1-t0)/10*1000:.2f} ms")

Max difference: 169.0122
Fast matmul: 175.51 ms
CuPy matmul: 456.13 ms


In [7]:
import numpy as np
import cupy as cp
from fast_matmul_wrapper import gpu_matmul
import time

# Print cupy configuration
print(f"cupy version: {cp.__version__}")
print(f"CUDA version: {cp.cuda.runtime.runtimeGetVersion()}")
print(f"Device: {cp.cuda.runtime.getDeviceProperties(0)['name'].decode()}")

# Try to get cuBLAS configuration
try:
    handle = cp.cuda.device.get_cublas_handle()
    print("cuBLAS handle obtained")
except:
    print("Could not get cuBLAS handle")

# Create test matrices; make sure not to normalise matricies or cupy will drop to 100Gflops!!
a = np.random.rand(5120, 5120).astype(np.float32)
b = np.random.rand(5120, 5120).astype(np.float32)

# Convert to cupy
a_cp = cp.asarray(a)
b_cp = cp.asarray(b)

# Function to run a single timed matmul
def timed_matmul(func, a, b, name=""):
    cp.cuda.Stream.null.synchronize()
    start = time.perf_counter()
    c = func(a, b)
    cp.cuda.Stream.null.synchronize()
    end = time.perf_counter()
    
    exec_time = end - start
    flops = 2.0 * float(5120**3)
    gflops = (flops / 1e9) / exec_time
    return gflops, c

# Try different ways of calling matmul
print("\nTesting different matmul approaches:")

# 1. Regular cupy matmul
gflops, c1 = timed_matmul(cp.matmul, a_cp, b_cp, "cupy.matmul")
print(f"cupy.matmul: {gflops:.2f} GFLOPS")

# 2. Our gpu_matmul
gflops, c2 = timed_matmul(gpu_matmul, a_cp, b_cp, "gpu_matmul")
print(f"gpu_matmul: {gflops:.2f} GFLOPS")

# Compare results
print(f"\nMax difference between implementations: {cp.max(cp.abs(c1 - c2))}")

# Now do full benchmark
def run_benchmark(func, name, warmup=5, runs=10):
    print(f"\nBenchmarking {name}...")
    
    # Warmup
    for _ in range(warmup):
        _ = func(a_cp, b_cp)
    cp.cuda.Stream.null.synchronize()
    
    # Timing runs
    times = []
    for i in range(runs):
        gflops, _ = timed_matmul(func, a_cp, b_cp)
        times.append(gflops)
        print(f"Run {i+1}: {gflops:.2f} GFLOPS")
    return times

# Run benchmarks
times_cupy = run_benchmark(cp.matmul, "cupy")
times_custom = run_benchmark(gpu_matmul, "gpu_matmul")

# Print statistics
def print_stats(name, times):
    print(f"\nPerformance Results for {name}:")
    print(f"  Minimum: {min(times):.2f} GFLOPS")
    print(f"  Maximum: {max(times):.2f} GFLOPS")
    print(f"  Average: {sum(times)/len(times):.2f} GFLOPS")
    print(f"  Std Dev: {np.std(times):.2f} GFLOPS")

print_stats("cupy", times_cupy)
print_stats("gpu_matmul", times_custom)


cupy version: 13.3.0
CUDA version: 12060
Device: NVIDIA RTX A1000 Laptop GPU
cuBLAS handle obtained

Testing different matmul approaches:
cupy.matmul: 4240.08 GFLOPS
gpu_matmul: 12288.51 GFLOPS

Max difference between implementations: 118.8031005859375

Benchmarking cupy...
Run 1: 4103.73 GFLOPS
Run 2: 4133.01 GFLOPS
Run 3: 4143.74 GFLOPS
Run 4: 4161.88 GFLOPS
Run 5: 4113.75 GFLOPS
Run 6: 4077.17 GFLOPS
Run 7: 4083.26 GFLOPS
Run 8: 3992.93 GFLOPS
Run 9: 3953.48 GFLOPS
Run 10: 3955.06 GFLOPS

Benchmarking gpu_matmul...
Run 1: 11983.20 GFLOPS
Run 2: 11986.29 GFLOPS
Run 3: 11982.33 GFLOPS
Run 4: 11984.53 GFLOPS
Run 5: 12006.31 GFLOPS
Run 6: 12046.60 GFLOPS
Run 7: 12075.94 GFLOPS
Run 8: 11935.67 GFLOPS
Run 9: 11939.36 GFLOPS
Run 10: 11926.43 GFLOPS

Performance Results for cupy:
  Minimum: 3953.48 GFLOPS
  Maximum: 4161.88 GFLOPS
  Average: 4071.80 GFLOPS
  Std Dev: 73.37 GFLOPS

Performance Results for gpu_matmul:
  Minimum: 11926.43 GFLOPS
  Maximum: 12075.94 GFLOPS
  Average: 11986.66 G

In [ ]:
# normalising values kills cupy performanc from 4,000gflops to only 100gflops!!
import numpy as np
import cupy as cp
import time

# Test both normal and normalized matrices
def test_both_versions():
    # Version 1: No normalization
    a1 = np.random.rand(5120, 5120).astype(np.float32)
    b1 = np.random.rand(5120, 5120).astype(np.float32)
    a1_cp = cp.asarray(a1)
    b1_cp = cp.asarray(b1)

    # Version 2: With normalization
    a2 = np.random.rand(5120, 5120).astype(np.float32)
    b2 = np.random.rand(5120, 5120).astype(np.float32)
    a2 = a2 / np.sqrt(a2.shape[1])
    b2 = b2 / np.sqrt(b2.shape[0])
    a2_cp = cp.asarray(a2)
    b2_cp = cp.asarray(b2)

    def benchmark(a, b, name):
        cp.cuda.Stream.null.synchronize()
        start = time.perf_counter()
        c = cp.matmul(a, b)
        cp.cuda.Stream.null.synchronize()
        end = time.perf_counter()
        
        exec_time = end - start
        flops = 2.0 * float(5120**3)
        gflops = (flops / 1e9) / exec_time
        return gflops, c

    print("Testing non-normalized matrices:")
    gflops1, c1 = benchmark(a1_cp, b1_cp, "non-normalized")
    print(f"GFLOPS: {gflops1:.2f}")
    print(f"Max value in result: {cp.max(cp.abs(c1))}")

    print("\nTesting normalized matrices:")
    gflops2, c2 = benchmark(a2_cp, b2_cp, "normalized")
    print(f"GFLOPS: {gflops2:.2f}")
    print(f"Max value in result: {cp.max(cp.abs(c2))}")

test_both_versions()


Testing non-normalized matrices:
GFLOPS: 3941.02
Max value in result: 1371.0440673828125

Testing normalized matrices:
GFLOPS: 106.10
Max value in result: 0.26630081527617694


The slowdown in the cuBLAS kernel is not as noticable (it is there but still running at 10Tflops).

In [2]:
import numpy as np
import cupy as cp
import time
from fast_matmul_wrapper import gpu_matmul

# Test both normal and normalized matrices
def test_both_versions():
    # Version 1: No normalization
    a1 = np.random.rand(5120, 5120).astype(np.float32)
    b1 = np.random.rand(5120, 5120).astype(np.float32)
    a1_cp = cp.asarray(a1)
    b1_cp = cp.asarray(b1)

    # Version 2: With normalization
    a2 = np.random.rand(5120, 5120).astype(np.float32)
    b2 = np.random.rand(5120, 5120).astype(np.float32)
    a2 = a2 / np.sqrt(a2.shape[1])
    b2 = b2 / np.sqrt(b2.shape[0])
    a2_cp = cp.asarray(a2)
    b2_cp = cp.asarray(b2)

    def benchmark_cupy(a, b):
        cp.cuda.Stream.null.synchronize()
        start = time.perf_counter()
        c = cp.matmul(a, b)
        cp.cuda.Stream.null.synchronize()
        end = time.perf_counter()
        
        exec_time = end - start
        flops = 2.0 * float(5120**3)
        gflops = (flops / 1e9) / exec_time
        return gflops, c

    def benchmark_gpu_matmul(a, b):
        cp.cuda.Stream.null.synchronize()
        start = time.perf_counter()
        c = gpu_matmul(a, b)
        cp.cuda.Stream.null.synchronize()
        end = time.perf_counter()
        
        exec_time = end - start
        flops = 2.0 * float(5120**3)
        gflops = (flops / 1e9) / exec_time
        return gflops, c

    # Test cupy.matmul
    print("Testing with cupy.matmul:")
    print("Non-normalized matrices:")
    gflops1, c1 = benchmark_cupy(a1_cp, b1_cp)
    print(f"GFLOPS: {gflops1:.2f}")
    print(f"Max value in result: {cp.max(cp.abs(c1))}")

    print("\nNormalized matrices:")
    gflops2, c2 = benchmark_cupy(a2_cp, b2_cp)
    print(f"GFLOPS: {gflops2:.2f}")
    print(f"Max value in result: {cp.max(cp.abs(c2))}")

    # Test gpu_matmul
    print("\nTesting with gpu_matmul:")
    print("Non-normalized matrices:")
    gflops3, c3 = benchmark_gpu_matmul(a1_cp, b1_cp)
    print(f"GFLOPS: {gflops3:.2f}")
    print(f"Max value in result: {cp.max(cp.abs(c3))}")

    print("\nNormalized matrices:")
    gflops4, c4 = benchmark_gpu_matmul(a2_cp, b2_cp)
    print(f"GFLOPS: {gflops4:.2f}")
    print(f"Max value in result: {cp.max(cp.abs(c4))}")

    # Verify results match between implementations
    print("\nVerifying results:")
    print("Non-normalized max difference:", cp.max(cp.abs(c1 - c3)))
    print("Normalized max difference:", cp.max(cp.abs(c2 - c4)))

test_both_versions()


Testing with cupy.matmul:
Non-normalized matrices:
GFLOPS: 4229.82
Max value in result: 1372.2125244140625

Normalized matrices:
GFLOPS: 106.35
Max value in result: 0.2683828339836041

Testing with gpu_matmul:
Non-normalized matrices:
GFLOPS: 14651.67
Max value in result: 1358.61376953125

Normalized matrices:
GFLOPS: 10492.84
Max value in result: 0.26880133152008057

Verifying results:
Non-normalized max difference: 122.855225
Normalized max difference: 0.024820299532119394
